In [ ]:
from google.colab import drive

# make Google drive available for use (dataset is saved there)
drive.mount("/content/gdrive", force_remount = True)

Mounted at /content/gdrive


In [ ]:
# extract dataset RAR file to session storage
!unrar x gdrive/MyDrive/NMDU/data.rar

Streaming output truncated to the last 5000 lines.
Extracting  data/valid/labels/00000_NewFire (3799).txt                    60%  OK 
Extracting  data/valid/labels/00000_NewFire (2481).txt                    60%  OK 
Extracting  data/valid/labels/NoFire (3127).txt                           60%  OK 
Extracting  data/valid/labels/00000_Fire 712.txt                          60%  OK 
Extracting  data/valid/labels/00000_NewFire (5991).txt                    60%  OK 
Extracting  data/valid/labels/NoFire (2489).txt                           60%  OK 
Extracting  data/valid/labels/NoFire (2374).txt                           60%  OK 
Extracting  data/valid/labels/NoFire (545).txt                            60%  OK 
Extracting  data/valid/labels/NoFire (583).txt                            60%  OK 
Extracting  data/valid/labels/NoFire (1852).txt                           60%  OK 
Extracting  data/valid/labels

In [ ]:
!pip install object-detection

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.9/121.9 kB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.9/84.9 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 20.6 MB/s eta 0:00:00


In [ ]:
import tensorflow as tf
from object_detection.utils import dataset_util
import os
import numpy as np
import cv2
import hashlib

In [ ]:
dataDir = "data"

# training directories
trainDir = os.path.join(dataDir, "train")
trainImagesDir = os.path.join(trainDir, "images")
trainLabelsDir = os.path.join(trainDir, "labels")

# validation directories
validDir = os.path.join(dataDir, "valid")
validImagesDir = os.path.join(validDir, "images")
validLabelsDir = os.path.join(validDir, "labels")

# testing directories
testDir = os.path.join(dataDir, "test")
testImagesDir = os.path.join(testDir, "images")
testLabelsDir = os.path.join(testDir, "labels")

print("Training directories:")
print(trainDir)
print(trainImagesDir)
print(trainLabelsDir)

print("\nValidation directories:")
print(validDir)
print(validImagesDir)
print(validLabelsDir)

print("\nTesting directories:")
print(testDir)
print(testImagesDir)
print(testLabelsDir)

Training directories:
data/train
data/train/images
data/train/labels

Validation directories:
data/valid
data/valid/images
data/valid/labels

Testing directories:
data/test
data/test/images
data/test/labels


In [ ]:
def preprocessImage(imagePath, labels):
  # targetSize = (224, 224)
  image = cv2.imread(imagePath)
  # image = cv2.resize(image, targetSize, interpolation = cv2.INTER_AREA)
  image = cv2.normalize(image, None, 0, 1.0, cv2.NORM_MINMAX, dtype=cv2.CV_32F)
  return image, labels

In [ ]:
def getImageBytes(imageDir, imageFilename):
  with tf.io.gfile.GFile(imageDir + "/" + imageFilename, 'rb') as fid:
    encodedImage = fid.read()
  key = hashlib.sha256(encodedImage).hexdigest()
  return encodedImage, key

In [ ]:
def createDataset(imagesDir, labelsDir):
  labels = []
  images = []
  imageFilenames = []

  for imageFilename in os.listdir(imagesDir):
    imageFilenames.append(imageFilename)
    labelFilename = os.path.splitext(imageFilename)[0] + ".txt"
    labelFilePath = os.path.join(labelsDir, labelFilename)
    with open(labelFilePath) as f:
      labelFileContent = f.read()
    label = []
    for line in labelFileContent.split("\n"):
      if len(line) > 0:
        label.append(np.array([float(x) for x in line.split()]))
    while len(label) < 3:
      label.append([1, -1, -1, -1, -1])
    l = np.array(label)
    labels.append(l)

    imagePath = os.path.join(imagesDir, imageFilename)
    img, _ = preprocessImage(imagePath, labels)
    images.append(img)

  return np.array(imageFilenames), np.array(images), np.array(labels)

In [ ]:
def generateTfrecord(imagesDirName, filename, image, label):
  height = image.shape[0]
  width = image.shape[0]
  image, key = getImageBytes(imagesDirName, filename)

  xmin = []
  ymin = []
  xmax = []
  ymax = []

  classesText = []
  classes = []

  if label[0][0] == 1:
      classesText.append(b"no_fire")
      classes.append(2)
      xmin.append(float(0))
      ymin.append(float(0))
      xmax.append(float(1))
      ymax.append(float(1))
  else:
    for labelItem in label:
      if labelItem[0] == 0:
        classesText.append(b"fire")
        classes.append(1)
        xmin.append(labelItem[1] - (labelItem[3] / 2))
        ymin.append(labelItem[2] - (labelItem[4] / 2))
        xmax.append(labelItem[1] + (labelItem[3] / 2))
        ymax.append(labelItem[2] + (labelItem[4] / 2))

  tfRecord = tf.train.Example(features=tf.train.Features(feature={
      'image/height': dataset_util.int64_feature(height),
      'image/width': dataset_util.int64_feature(width),
      'image/filename': dataset_util.bytes_feature(filename.encode('utf8')),
      'image/source_id': dataset_util.bytes_feature(filename.encode('utf8')),
      'image/key/sha256': dataset_util.bytes_feature(key.encode('utf8')),
      'image/encoded': dataset_util.bytes_feature(image),
      'image/format': dataset_util.bytes_feature('jpg'.encode('utf8')),
      'image/object/bbox/xmin': dataset_util.float_list_feature(xmin),
      'image/object/bbox/xmax': dataset_util.float_list_feature(xmax),
      'image/object/bbox/ymin': dataset_util.float_list_feature(ymin),
      'image/object/bbox/ymax': dataset_util.float_list_feature(ymax),
      'image/object/class/text': dataset_util.bytes_list_feature(classesText),
      'image/object/class/label': dataset_util.int64_list_feature(classes)
  }))
  return tfRecord

In [ ]:
trainTfRecordFilename = "train.tfrecord"
validTfRecordFilename = "valid.tfrecord"
testTfRecordFilename = "test.tfrecord"

In [ ]:
writerTrain = tf.io.TFRecordWriter(trainTfRecordFilename)
filenamesTrain, imagesTrain, labelsTrain = createDataset(trainImagesDir, trainLabelsDir)

for filenameTrain, imageTrain, labelTrain in zip(filenamesTrain, imagesTrain, labelsTrain):
  tfRecordTrain = generateTfrecord(trainImagesDir, filenameTrain, imageTrain, labelTrain)
  writerTrain.write(tfRecordTrain.SerializeToString())

writerTrain.close()

In [ ]:
writerValid = tf.io.TFRecordWriter(validTfRecordFilename)
filenamesValid, imagesValid, labelsValid = createDataset(validImagesDir, validLabelsDir)

for filenameValid, imageValid, labelValid in zip(filenamesValid, imagesValid, labelsValid):
  tfRecordValid = generateTfrecord(validImagesDir, filenameValid, imageValid, labelValid)
  writerValid.write(tfRecordValid.SerializeToString())

writerValid.close()

In [ ]:
writerTest = tf.io.TFRecordWriter(testTfRecordFilename)
filenamesTest, imagesTest, labelsTest = createDataset(testImagesDir, testLabelsDir)

for filenameTest, imageTest, labelTest in zip(filenamesTest, imagesTest, labelsTest):
  tfRecordTest = generateTfrecord(testImagesDir, filenameTest, imageTest, labelTest)
  writerTest.write(tfRecordTest.SerializeToString())

writerTest.close()

In [ ]:
!cp train.tfrecord valid.tfrecord test.tfrecord /content/gdrive/MyDrive/NMDU/tfRecords/

In [ ]:
!pip install pip==21.3.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 9.9 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pip-tools 6.13.0 requires pip>=22.2, but you have pip 21.3.1 which is incompatible.


In [ ]:
!git clone https://github.com/tensorflow/models.git

Cloning into 'models'...
remote: Enumerating objects: 86791, done.
remote: Counting objects: 100% (545/545), done.
remote: Compressing objects: 100% (263/263), done.
remote: Total 86791 (delta 291), reused 510 (delta 276), pack-reused 86246
Receiving objects: 100% (86791/86791), 598.93 MiB | 22.56 MiB/s, done.
Resolving deltas: 100% (62173/62173), done.


In [ ]:
%cd models/research

/content/models/research


In [ ]:
!protoc object_detection/protos/*.proto --python_out=.
# Install TensorFlow Object Detection API.
!cp object_detection/packages/tf2/setup.py .
!python -m pip install .

Processing /content/models/research
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     |████████████████████████████████| 14.6 MB 950 kB/s            
     |████████████████████████████████| 2.6 MB 64.7 MB/s            
     |████████████████████████████████| 28.0 MB 101.0 MB/s            
     |████████████████████████████████| 67 kB 6.4 MB/s             
     |████████████████████████████████| 116 kB 75.5 MB/s            
     |████████████████████████████████| 1.3 MB 22.5 MB/s            
     |████████████████████████████████| 524.1 MB 23 kB/s             
     |████████████████████████████████| 6.5 MB 69.1 MB/s            
     |████████████████████████████████| 269 kB 67.0 MB/s            
  Preparing metadata (setup.py) ... done
     |████████████████████████████████| 43 kB 1.7 MB/s             
  Preparing metadata (setup.py) ... done
     |████████████████████████████████| 241 kB 63.1 MB/s            
     |████████████████████████████████| 

In [ ]:
!python object_detection/builders/model_builder_tf2_test.py

2023-07-25 08:01:40.093942: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/usr/local/lib/python3.10/dist-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/usr/local/lib/python3.10/dist-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/usr/local/lib/python3.10/dist-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6Status12empty_stringB5cxx11Ev']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/usr/local/lib/python3.10/dist-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/usr/local/lib/python3.10/dist-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/usr/local/lib/python3.10/dist-packages/tensorflow_io/pytho

In [ ]:
%cd /content

/content


In [ ]:
!wget http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz

!tar -xzvf ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz

--2023-07-25 08:06:19--  http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz
Resolving download.tensorflow.org (download.tensorflow.org)... 142.251.2.128, 2607:f8b0:4023:c0d::80
Connecting to download.tensorflow.org (download.tensorflow.org)|142.251.2.128|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 20515344 (20M) [application/x-tar]
Saving to: ‘ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz’

ssd_mobilenet_v2_fp 100%[===================>]  19.56M  63.3MB/s    in 0.3s    

2023-07-25 08:06:19 (63.3 MB/s) - ‘ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz’ saved [20515344/20515344]

ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/
ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/checkpoint/
ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/checkpoint/ckpt-0.data-00000-of-00001
ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/checkpoint/checkpoint
ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/

In [ ]:
!wget http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_320x320_coco17_tpu-8.tar.gz

!tar -xzvf ssd_mobilenet_v2_320x320_coco17_tpu-8.tar.gz

--2023-07-25 08:03:57--  http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_320x320_coco17_tpu-8.tar.gz
Resolving download.tensorflow.org (download.tensorflow.org)... 142.251.2.128, 2607:f8b0:4023:c0d::80
Connecting to download.tensorflow.org (download.tensorflow.org)|142.251.2.128|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 46042990 (44M) [application/x-tar]
Saving to: ‘ssd_mobilenet_v2_320x320_coco17_tpu-8.tar.gz’

ssd_mobilenet_v2_32 100%[===================>]  43.91M  65.4MB/s    in 0.7s    

2023-07-25 08:03:58 (65.4 MB/s) - ‘ssd_mobilenet_v2_320x320_coco17_tpu-8.tar.gz’ saved [46042990/46042990]

ssd_mobilenet_v2_320x320_coco17_tpu-8/
ssd_mobilenet_v2_320x320_coco17_tpu-8/checkpoint/
ssd_mobilenet_v2_320x320_coco17_tpu-8/checkpoint/ckpt-0.data-00000-of-00001
ssd_mobilenet_v2_320x320_coco17_tpu-8/checkpoint/checkpoint
ssd_mobilenet_v2_320x320_coco17_tpu-8/checkpoint/ckpt-0.index
ssd_mobilenet_v2_320x320_coco17_tpu-8/pi

In [ ]:
!wget http://download.tensorflow.org/models/object_detection/tf2/20200711/faster_rcnn_resnet50_v1_640x640_coco17_tpu-8.tar.gz

!tar -xzvf faster_rcnn_resnet50_v1_640x640_coco17_tpu-8.tar.gz

--2023-07-19 11:05:15--  http://download.tensorflow.org/models/object_detection/tf2/20200711/faster_rcnn_resnet50_v1_640x640_coco17_tpu-8.tar.gz
Resolving download.tensorflow.org (download.tensorflow.org)... 142.251.167.128, 2607:f8b0:4004:c1d::80
Connecting to download.tensorflow.org (download.tensorflow.org)|142.251.167.128|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 211996178 (202M) [application/x-tar]
Saving to: ‘faster_rcnn_resnet50_v1_640x640_coco17_tpu-8.tar.gz’

faster_rcnn_resnet5 100%[===================>] 202.17M   176MB/s    in 1.1s    

2023-07-19 11:05:17 (176 MB/s) - ‘faster_rcnn_resnet50_v1_640x640_coco17_tpu-8.tar.gz’ saved [211996178/211996178]

faster_rcnn_resnet50_v1_640x640_coco17_tpu-8/
faster_rcnn_resnet50_v1_640x640_coco17_tpu-8/checkpoint/
faster_rcnn_resnet50_v1_640x640_coco17_tpu-8/checkpoint/ckpt-0.data-00000-of-00001
faster_rcnn_resnet50_v1_640x640_coco17_tpu-8/checkpoint/checkpoint
faster_rcnn_resnet50_v1_640x640_coco17_tpu-8/c

In [ ]:
!wget http://download.tensorflow.org/models/object_detection/tf2/20200713/centernet_hg104_512x512_coco17_tpu-8.tar.gz

!tar -xzvf centernet_hg104_512x512_coco17_tpu-8.tar.gz

--2023-07-19 14:42:20--  http://download.tensorflow.org/models/object_detection/tf2/20200713/centernet_hg104_512x512_coco17_tpu-8.tar.gz
Resolving download.tensorflow.org (download.tensorflow.org)... 142.251.6.128, 2607:f8b0:4001:c5a::80
Connecting to download.tensorflow.org (download.tensorflow.org)|142.251.6.128|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1426099846 (1.3G) [application/x-tar]
Saving to: ‘centernet_hg104_512x512_coco17_tpu-8.tar.gz’

centernet_hg104_512 100%[===================>]   1.33G   126MB/s    in 12s     

2023-07-19 14:42:32 (114 MB/s) - ‘centernet_hg104_512x512_coco17_tpu-8.tar.gz’ saved [1426099846/1426099846]

centernet_hg104_512x512_coco17_tpu-8/
centernet_hg104_512x512_coco17_tpu-8/checkpoint/
centernet_hg104_512x512_coco17_tpu-8/checkpoint/ckpt-0.data-00000-of-00001
centernet_hg104_512x512_coco17_tpu-8/checkpoint/checkpoint
centernet_hg104_512x512_coco17_tpu-8/checkpoint/ckpt-0.index
centernet_hg104_512x512_coco17_tpu-8/pipel

In [ ]:
!apt-get install rar

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  rar
0 upgraded, 1 newly installed, 0 to remove and 8 not upgraded.
Need to get 341 kB of archives.
After this operation, 1,202 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/multiverse amd64 rar amd64 2:5.5.0-1.1 [341 kB]
Fetched 341 kB in 1s (355 kB/s)
Selecting previously unselected package rar.
(Reading database ... 129824 files and directories currently installed.)
Preparing to unpack .../rar_2%3a5.5.0-1.1_amd64.deb ...
Unpacking rar (2:5.5.0-1.1) ...
Setting up rar (2:5.5.0-1.1) ...
Processing triggers for man-db (2.10.2-1) ...


In [ ]:
!rar a "/content/gdrive/MyDrive/NMDU/modelsOriginal/modelSsdMobileNetFpnLite.rar" "ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8"


RAR 5.50   Copyright (c) 1993-2017 Alexander Roshal   11 Aug 2017
Trial version             Type 'rar -?' for help

Evaluation copy. Please register.

Creating archive /content/gdrive/MyDrive/NMDU/modelsOriginal/modelSsdMobileNetFpnLite.rar

Adding    ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/saved_model/saved_model.pb      13% 26% 30%  OK 
Adding    ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/saved_model/variables/variables.index      30%  OK 
Adding    ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/saved_model/variables/variables.data-00000-of-00001      44% 57% 65%  OK 
Adding    ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/checkpoint/checkpoint      65%  OK 
Adding    ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/checkpoint/ckpt-0.index      65%  OK 
Adding    ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/checkpoint/ckpt-0.data-00000-of-00001      78% 92% 99%  OK 
Adding    ssd_mobilenet_v2_fpnlite

In [ ]:
!rar a "/content/gdrive/MyDrive/NMDU/modelsOriginal/modelSsdMobileNet.rar" "ssd_mobilenet_v2_320x320_coco17_tpu-8"


RAR 5.50   Copyright (c) 1993-2017 Alexander Roshal   11 Aug 2017
Trial version             Type 'rar -?' for help

Evaluation copy. Please register.

Creating archive /content/gdrive/MyDrive/NMDU/modelsOriginal/modelSsdMobileNet.rar

Adding    ssd_mobilenet_v2_320x320_coco17_tpu-8/saved_model/saved_model.pb       7% 14% 15%  OK 
Adding    ssd_mobilenet_v2_320x320_coco17_tpu-8/saved_model/variables/variables.index      15%  OK 
Adding    ssd_mobilenet_v2_320x320_coco17_tpu-8/saved_model/variables/variables.data-00000-of-00001      22% 30% 37% 44% 51% 57%  OK 
Adding    ssd_mobilenet_v2_320x320_coco17_tpu-8/checkpoint/checkpoint      57%  OK 
Adding    ssd_mobilenet_v2_320x320_coco17_tpu-8/checkpoint/ckpt-0.index      57%  OK 
Adding    ssd_mobilenet_v2_320x320_coco17_tpu-8/checkpoint/ckpt-0.data-00000-of-00001      65% 72% 79% 86% 93% 99%  OK 
Adding    ssd_mobilenet_v2_320x320_coco17

In [ ]:
!rar a "/content/gdrive/MyDrive/NMDU/modelsOriginal/modelFasterRcnnResNet50.rar" "faster_rcnn_resnet50_v1_640x640_coco17_tpu-8"


RAR 5.50   Copyright (c) 1993-2017 Alexander Roshal   11 Aug 2017
Trial version             Type 'rar -?' for help

Evaluation copy. Please register.

Creating archive /content/gdrive/MyDrive/NMDU/modelsOriginal/modelFasterRcnnResNet50.rar

Adding    faster_rcnn_resnet50_v1_640x640_coco17_tpu-8/checkpoint/checkpoint       0%  OK 
Adding    faster_rcnn_resnet50_v1_640x640_coco17_tpu-8/checkpoint/ckpt-0.index       0%  OK 
Adding    faster_rcnn_resnet50_v1_640x640_coco17_tpu-8/checkpoint/ckpt-0.data-00000-of-00001       1%  3%  5%  7%  8% 10% 12% 14% 16% 17% 19% 21% 23% 24% 26% 28% 30% 32% 33% 35% 37% 39% 40% 42% 44% 46% 48%  OK 
Adding    faster_rcnn_resnet50_v1_640x640_coco17_tpu-8/pipeline.config      48%  OK 
Adding    faster_rcnn_resnet50_v1_640x640_coco17_tpu-8/saved_model/variables/variables.data-00000-of-00001      50% 52% 53%

In [ ]:
!rar a "/content/gdrive/MyDrive/NMDU/modelsOriginal/modelCenterNetHourglass.rar" "centernet_hg104_512x512_coco17_tpu-8"


RAR 5.50   Copyright (c) 1993-2017 Alexander Roshal   11 Aug 2017
Trial version             Type 'rar -?' for help

Evaluation copy. Please register.

Creating archive /content/gdrive/MyDrive/NMDU/modelsOriginal/modelCenterNetHourglass.rar

Adding    centernet_hg104_512x512_coco17_tpu-8/checkpoint/checkpoint       0%  OK 
Adding    centernet_hg104_512x512_coco17_tpu-8/checkpoint/ckpt-0.index       0%  OK 
Adding    centernet_hg104_512x512_coco17_tpu-8/checkpoint/ckpt-0.data-00000-of-00001       0%  1%  2%  3%  4%  5%  6%  7%  8%  9% 10% 11% 12% 13% 14% 15% 16% 17% 18% 19% 20% 21% 22% 23% 24% 25% 26% 27% 28% 29% 30% 31% 32% 33% 34% 35% 36% 37% 38% 39% 40% 41% 42% 43% 44% 45% 46% 47% 48%  OK 
Adding    centernet_hg104_512x512_coco17_tpu-8/pipeline.config            48

In [ ]:
!unrar x /content/gdrive/MyDrive/NMDU/modelsOriginal/modelSsdMobileNetFpnLite.rar /content/gdrive/MyDrive/NMDU/modelsInitial


UNRAR 6.11 beta 1 freeware      Copyright (c) 1993-2022 Alexander Roshal


Extracting from /content/gdrive/MyDrive/NMDU/modelsOriginal/modelSsdMobileNetFpnLite.rar

Creating    /content/gdrive/MyDrive/NMDU/modelsInitial/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8  OK
Creating    /content/gdrive/MyDrive/NMDU/modelsInitial/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/saved_model  OK
Extracting  /content/gdrive/MyDrive/NMDU/modelsInitial/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/saved_model/saved_model.pb       1%  OK 
Creating    /content/gdrive/MyDrive/NMDU/modelsInitial/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/saved_model/variables  OK
Extracting  /content/gdrive/MyDrive/NMDU/modelsInitial/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/saved_model/variables/variables.index       1%  OK 
Extracting  /content/gdrive/MyDrive/NMDU/modelsInitial/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/saved_model/variables/variables.data-00000-of-00001      21% 42

In [ ]:
!unrar x /content/gdrive/MyDrive/NMDU/modelsOriginal/modelSsdMobileNet.rar /content/gdrive/MyDrive/NMDU/modelsInitial


UNRAR 6.11 beta 1 freeware      Copyright (c) 1993-2022 Alexander Roshal


Extracting from /content/gdrive/MyDrive/NMDU/modelsOriginal/modelSsdMobileNet.rar

Creating    /content/gdrive/MyDrive/NMDU/modelsInitial/ssd_mobilenet_v2_320x320_coco17_tpu-8  OK
Creating    /content/gdrive/MyDrive/NMDU/modelsInitial/ssd_mobilenet_v2_320x320_coco17_tpu-8/saved_model  OK
Extracting  /content/gdrive/MyDrive/NMDU/modelsInitial/ssd_mobilenet_v2_320x320_coco17_tpu-8/saved_model/saved_model.pb       0%  OK 
Creating    /content/gdrive/MyDrive/NMDU/modelsInitial/ssd_mobilenet_v2_320x320_coco17_tpu-8/saved_model/variables  OK
Extracting  /content/gdrive/MyDrive/NMDU/modelsInitial/ssd_mobilenet_v2_320x320_coco17_tpu-8/saved_model/variables/variables.index       0%  OK 
Extracting  /content/gdrive/MyDrive/NMDU/modelsInitial/ssd_mobilenet_v2_320x320_coco17_tpu-8/saved_model/variables/variables.data-00000-of-00001       9% 18% 27% 37% 46% 50%  OK 
Creating   

In [ ]:
!unrar x /content/gdrive/MyDrive/NMDU/modelsOriginal/modelFasterRcnnResNet50.rar /content/gdrive/MyDrive/NMDU/modelsInitial


UNRAR 6.11 beta 1 freeware      Copyright (c) 1993-2022 Alexander Roshal


Extracting from /content/gdrive/MyDrive/NMDU/modelsOriginal/modelFasterRcnnResNet50.rar

Creating    /content/gdrive/MyDrive/NMDU/modelsInitial/faster_rcnn_resnet50_v1_640x640_coco17_tpu-8  OK
Creating    /content/gdrive/MyDrive/NMDU/modelsInitial/faster_rcnn_resnet50_v1_640x640_coco17_tpu-8/checkpoint  OK
Extracting  /content/gdrive/MyDrive/NMDU/modelsInitial/faster_rcnn_resnet50_v1_640x640_coco17_tpu-8/checkpoint/checkpoint       0%  OK 
Extracting  /content/gdrive/MyDrive/NMDU/modelsInitial/faster_rcnn_resnet50_v1_640x640_coco17_tpu-8/checkpoint/ckpt-0.index       0%  OK 
Extracting  /content/gdrive/MyDrive/NMDU/modelsInitial/faster_rcnn_resnet50_v1_640x640_coco17_tpu-8/checkpoint/ckpt-0.data-00000-of-00001       1%  3%  5%  7%  9% 11% 13% 15% 17% 19% 21% 23% 25% 27% 29% 31% 33% 35% 37% 39% 41% 43% 4

In [ ]:
!unrar x /content/gdrive/MyDrive/NMDU/modelsOriginal/modelCenterNetHourglass.rar /content/gdrive/MyDrive/NMDU/modelsInitial


UNRAR 6.11 beta 1 freeware      Copyright (c) 1993-2022 Alexander Roshal


Extracting from /content/gdrive/MyDrive/NMDU/modelsOriginal/modelCenterNetHourglass.rar

Creating    /content/gdrive/MyDrive/NMDU/modelsInitial/centernet_hg104_512x512_coco17_tpu-8  OK
Creating    /content/gdrive/MyDrive/NMDU/modelsInitial/centernet_hg104_512x512_coco17_tpu-8/checkpoint  OK
Extracting  /content/gdrive/MyDrive/NMDU/modelsInitial/centernet_hg104_512x512_coco17_tpu-8/checkpoint/checkpoint       0%  OK 
Extracting  /content/gdrive/MyDrive/NMDU/modelsInitial/centernet_hg104_512x512_coco17_tpu-8/checkpoint/ckpt-0.index       0%  OK 
Extracting  /content/gdrive/MyDrive/NMDU/modelsInitial/centernet_hg104_512x512_coco17_tpu-8/checkpoint/ckpt-0.data-00000-of-00001       0%  1%  2%  3%  4%  5%  6%  7%  8%  9% 10% 11% 12% 13% 14% 15% 16% 17% 18% 19% 20% 21% 22% 23% 24% 25% 26% 2

In [ ]:
!cp /content/models/research/object_detection/configs/tf2/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.config \
  /content/models/research/object_detection/configs/tf2/ssd_mobilenet_v2_320x320_coco17_tpu-8.config \
  /content/models/research/object_detection/configs/tf2/faster_rcnn_resnet50_v1_640x640_coco17_tpu-8.config \
  /content/models/research/object_detection/configs/tf2/centernet_hourglass104_512x512_coco17_tpu-8.config \
  /content/

In [ ]:
!cp ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.config \
  ssd_mobilenet_v2_320x320_coco17_tpu-8.config \
  faster_rcnn_resnet50_v1_640x640_coco17_tpu-8.config \
  centernet_hourglass104_512x512_coco17_tpu-8.config \
  /content/gdrive/MyDrive/NMDU/configsOriginal/

PROMJENE CONFIG FILE-OVA:

ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.config
- model ssd num_classes
- train_input_reader label_map_path
- train_input_reader tf_record_input_reader input_path
- eval_input_reader label_map_path
- eval_input_reader tf_record_input_reader input_path
- train_config fine_tune_checkpoint
- train_config fine_tune_checkpoint_type
- train_config optimizer momentum_optimizer learning_rate cosine_decay_learning_rate total_steps
- train_config optimizer momentum_optimizer learning_rate cosine_decay_learning_rate warmup_steps
- train_config batch_size
- train_config num_steps

ssd_mobilenet_v2_320x320_coco17_tpu-8.config
- model ssd num_classes
- train_input_reader label_map_path
- train_input_reader tf_record_input_reader input_path
- eval_input_reader label_map_path
- eval_input_reader tf_record_input_reader input_path
- train_config fine_tune_checkpoint
- train_config fine_tune_checkpoint_type
- train_config optimizer momentum_optimizer learning_rate cosine_decay_learning_rate total_steps
- train_config optimizer momentum_optimizer learning_rate cosine_decay_learning_rate warmup_steps
- train_config batch_size
- train_config num_steps

faster_rcnn_resnet50_v1_640x640_coco17_tpu-8.config
- model faster_rcnn num_classes
- train_input_reader label_map_path
- train_input_reader tf_record_input_reader input_path
- eval_input_reader label_map_path
- eval_input_reader tf_record_input_reader input_path
- train_config fine_tune_checkpoint
- train_config fine_tune_checkpoint_type
- train_config optimizer momentum_optimizer learning_rate cosine_decay_learning_rate total_steps
- train_config optimizer momentum_optimizer learning_rate cosine_decay_learning_rate warmup_steps
- train_config batch_size
- train_config num_steps

centernet_hourglass104_512x512_coco17_tpu-8.config
- model center_net num_classes
- train_input_reader label_map_path
- train_input_reader tf_record_input_reader input_path
- eval_input_reader label_map_path
- eval_input_reader tf_record_input_reader input_path
- train_config fine_tune_checkpoint
- train_config optimizer adam_optimizer learning_rate manual_step_learning_rate schedule step
- train_config optimizer adam_optimizer learning_rate manual_step_learning_rate schedule step
- train_config batch_size
- train_config num_steps

In [ ]:
!cp ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.config \
  ssd_mobilenet_v2_320x320_coco17_tpu-8.config \
  faster_rcnn_resnet50_v1_640x640_coco17_tpu-8.config \
  centernet_hourglass104_512x512_coco17_tpu-8.config \
  /content/gdrive/MyDrive/NMDU/configsInitial/